In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
def rgb_to_gray(rgb):
    # use numpy function np.dot() to calculate the dot product of two arrays
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.114])

def most_freq_pixel_of_neighbors(image, x, y, half_of_window_size):
    # given an column index and row index and the window size, we can find the current window
    current_window = image[x-half_of_window_size:x+half_of_window_size+1,y-half_of_window_size:y+half_of_window_size+1]
    # use histogram calculation to get the most freqent pixel value
    histogram = cv2.calcHist([current_window],[0], None,[256],[0,256])
    max_of_histogram = np.max(histogram)
    # returning the index of the most frequent pixel, ie. the value
    return np.argwhere(histogram == max_of_histogram)[0][0]

def set_pixel_to_most_freq_pixels(image, window_size):
    # cut the window size to half
    half_of_window_size = window_size // 2
    # add the border 
    img_reflect_101 = cv2.copyMakeBorder(image,half_of_window_size,half_of_window_size,half_of_window_size,half_of_window_size,cv2.BORDER_REFLECT_101)
    # use a nest-loop to set every pixel to a new value
    for x in range(image.shape[0]):
        for y in range(image.shape[1]):
            image.itemset((x,y),most_freq_pixel_of_neighbors(img_reflect_101,x+half_of_window_size,y+half_of_window_size,half_of_window_size))
            
def get_color_indensities(B, J, x, y, half_of_window_size):
    # current value of pixel of coordinates (x,y)
    current_pixel_value = J.item(x,y)
    # create an empty list with three empty lists inside to store rgb values
    rgb_values = [[] for _ in range(3)]
    for i in range(x-half_of_window_size,x+half_of_window_size+1):
        # check the border
        if i >= 0 and i < J.shape[0]:
            for j in range(y-half_of_window_size,y+half_of_window_size+1):
                # check the border and check the same pixel value
                if j >= 0 and j < J.shape[1] and J.item(i,j) == current_pixel_value:
                    rgb_values[0].append(B.item(i,j,0))
                    rgb_values[1].append(B.item(i,j,1))
                    rgb_values[2].append(B.item(i,j,2))
    # calculate the average intensities of those pixels in each band
    return (sum(rgb_values[0])/len(rgb_values[0]),sum(rgb_values[1])/len(rgb_values[1]),sum(rgb_values[2])/len(rgb_values[2]))

def generate_painting(B,J,window_size):
    # cut the window size to half
    half_of_window_size = window_size//2
    # Allocate these variables, so the program 
    row_size, col_size = J.shape
    # use a nest-loop the set the pixel values
    for x in range(0,row_size):
        for y in range(0,col_size):
            B[x,y,:] = get_color_indensities(B,J,x,y,half_of_window_size)

# Variables Need to Be Changed

In [ ]:
file_name = 'dog.jpg'
image_name = file_name.split('.')[0]

# Task 1

In [ ]:
# read image and convert BGR to RGB
A = cv2.imread('dog.jpg', 1)
A = cv2.cvtColor(A, cv2.COLOR_BGR2RGB)
# apply the transform function to transform RGB to grayscale
I = rgb_to_gray(A)
# set the dtype of image I to be uint8
I = I.astype(np.uint8)
# write to task_1.jpg
cv2.imwrite(f'{image_name}_task_1.jpg',I)

# Task 2

In [ ]:
# make a copy of I
J = np.copy(I)
# put the three window sizes into a list and write them to images formatted
window_size_list = [3, 20, 50]
# loop these three window sizes
for window_size in window_size_list:
    set_pixel_to_most_freq_pixels(J,window_size)
    # write to formatted names
    cv2.imwrite(f'{image_name}_task_2_{window_size}.jpg', J)

# Task 3

In [ ]:
# we will use the same window sizes as Task 2, and loop these three window sizes
for window_size in window_size_list:
    B = np.copy(A)
    generate_painting(B,J,window_size)
    # convert them to BGR
    B = cv2.cvtColor(B, cv2.COLOR_RGB2BGR)
    # write to formatted names 
    cv2.imwrite(f'{image_name}_task_3_{window_size}.jpg', B)